In [2]:
import dowhy
from dowhy import CausalModel

import numpy as np
import pandas as pd
import graphviz
import networkx as nx

np.set_printoptions(precision=3, suppress=True)
np.random.seed(0)
# Import necessary libraries
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import dowhy
from dowhy import CausalModel
import re
from collections import Counter
import pandas as pd
from sklearn.preprocessing import StandardScaler
from dowhy import CausalModel



def load_data(filepath):
    df = pd.read_csv(filepath, header=None, skiprows=0)
    df.columns = df.iloc[0]
    df = df[1:]
    return df



# Load and preprocess data
file_path = r"C:\Users\nstep\TSU\CLEANED\df_selected1.csv"
nashvilleDF = load_data(file_path)

#nashvilleDf to lower case

nashvilleDF.head()

#check to see if nashvilleDF has a column named 'id' and show results for 5 rows
nashvilleDF['id'].head()
#drop 'id' from nashvilleDF
nashvilleDF = nashvilleDF.drop('id', axis=1)

# Drop rows where any of the specified columns has a value of 1
columns_to_check = ['prop_Entire_condo', 'prop_Entire_guest_suite', 'prop_Entire_guesthouse',
                        'prop_Entire_rental_unit', 'prop_Entire_townhouse', 'prop_Hotel',
                        'prop_Private_room', 'free_parking', 'room_Privat_room']

nashvilleDF = nashvilleDF[~nashvilleDF[columns_to_check].eq(1).any(axis=1)]

# Drop 'id' and the specified columns
columns_to_drop = ['prop_Entire_home', 'room_Entire_home/apt',] + columns_to_check
nashvilleDF = nashvilleDF.drop(columns=columns_to_drop)
nashvilleDF.head()


,grill,private_entrance,review_scores_location,resort_access,review_scores_rating,bedrooms,beds,pool,host_acceptance_rate,reviews_per_month,minimum_maximum_nights,minimum_minimum_nights,bathrooms,neighbourhood_cleansed_num,accommodates,hot_tub,fireplace,price,host_is_superhost
1,0,0,4.92,0,4.95,1,1,0,0,3.91,365,30,1,6,1,0,0,43,0
2,0,0,4.94,0,4.94,1,1,0,93,4.52,1125,1,1,25,2,0,0,70,0
3,0,0,4.96,0,4.91,2,2,0,100,4.71,1125,2,1,18,2,0,0,109,1
4,0,0,4.74,0,4.77,1,2,0,91,0.68,365,30,1,12,2,0,2,39,0
5,0,0,4.47,0,4.71,1,3,0,91,0.54,365,30,1,12,1,1,1,33,0


In [3]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import dowhy
from dowhy import CausalModel




common_causes_dict = {
    ("private_entrance", "resort_access"): [],
    ("private_entrance", "price"): [],
    ("review_scores_location", "review_scores_rating"): ["neighbourhood_cleansed_num"],
    ("resort_access", "pool"): [],
    ("resort_access", "price"): ["private_entrance"],
    ("resort_access", "hot_tub"): [],
    ("review_scores_rating", "reviews_per_month"): ["host_is_superhost"],
    ("accommodates", "bathrooms"): [],
    ("bedrooms", "beds"): [],
    ("bedrooms", "price"): ["accommodates"],
    ("bathrooms", "price"): ["accommodates"],
    ("grill", "price"): [],
    ("pool", "host_acceptance_rate"): ["resort_access"],
    ("pool", "price"): ["resort_access"],
    ("host_acceptance_rate", "reviews_per_month"): [],
    ("host_acceptance_rate", "price"): [],
    ("reviews_per_month", "price"): ["review_scores_rating", "host_acceptance_rate"],
    ("minimum_maximum_nights", "reviews_per_month"): [],
    ("minimum_maximum_nights", "price"): [],
    ("bedrooms", "accommodates"): [],
    ("beds", "accommodates"): ["bedrooms"],
    ("neighbourhood_cleansed_num", "review_scores_location"): [],
    ("neighbourhood_cleansed_num", "price"): [],
    ("accommodates", "price"): ["bedrooms", "bathrooms"],
    ("hot_tub", "pool"): ["resort_access"],
    ("hot_tub", "host_acceptance_rate"): ["resort_access"],
    ("fireplace", "price"): [],
    ("host_is_superhost", "review_scores_rating"): [],
    ("host_is_superhost", "reviews_per_month"): [],
}


variable_pairs = [
    ("private_entrance", "resort_access"),
    ("private_entrance", "price"),
    ("review_scores_location", "review_scores_rating"),
    ("resort_access", "pool"),
    ("resort_access", "price"),
    ("resort_access", "hot_tub"),
    ("review_scores_rating", "reviews_per_month"),
    ("accommodates", "bathrooms"),
    ("bedrooms", "beds"),
    ("bedrooms", "price"),
    ("bathrooms", "price"),
    ("grill", "price"),
    ("pool", "host_acceptance_rate"),
    ("pool", "price"),
    ("host_acceptance_rate", "reviews_per_month"),
    ("host_acceptance_rate", "price"),
    ("reviews_per_month", "price"),
    ("minimum_maximum_nights", "reviews_per_month"),
    ("minimum_maximum_nights", "price"),
    ("bedrooms", "accommodates"),
    ("beds", "accommodates"),
    ("neighbourhood_cleansed_num", "review_scores_location"),
    ("neighbourhood_cleansed_num", "price"),
    ("accommodates", "price"),
    ("hot_tub", "pool"),
    ("hot_tub", "host_acceptance_rate"),
    ("fireplace", "price"),
    ("host_is_superhost", "review_scores_rating"),
    ("host_is_superhost", "reviews_per_month"),
]




# Preprocessing
scaler = StandardScaler()
nashvilleDF_scaled = pd.DataFrame(scaler.fit_transform(nashvilleDF), columns=nashvilleDF.columns)

df = nashvilleDF_scaled.copy()

# Assuming 'data' is a pandas DataFrame containing your Airbnb data.

# Specifying common causes for each variable pair based on the DAG.


# Store the results in dictionaries
causal_effects = {}
correlation_scores = {}

for cause, effect in variable_pairs:
    common_causes = common_causes_dict.get((cause, effect), [])

    # Define the causal model
    model = CausalModel(
        data=df,
        treatment=cause,
        outcome=effect,
        common_causes=common_causes
    )
    
    # Identify the causal effect
    identified_estimand = model.identify_effect()

    # Estimate the causal effect using linear regression
    causal_estimate = model.estimate_effect(identified_estimand,
                                            method_name="backdoor.linear_regression")

    # Store the causal estimate in the results dictionary
    causal_effects[(cause, effect)] = causal_estimate.value

    # Calculate the correlation
    correlation_score, _ = pearsonr(df[cause], df[effect])
    correlation_scores[(cause, effect)] = correlation_score

# Output the results
for (cause, effect), estimate in causal_effects.items():
    print(f"Causal Effect of {cause} on {effect}: {estimate}")

# For correlation scores, if you just want the keys and values:
correlation_keys = list(correlation_scores.keys())
correlation_values = list(correlation_scores.values())

# If you need to display the correlation scores
print("\nCorrelation Scores:")
for pair, score in correlation_scores.items():
    print(f"Correlation between {pair[0]} and {pair[1]}: {score}")

# Extracting the values from the causal_effects dictionary
causation_values = list(causal_effects.values())

# If you need to display the causation scores
print("\nCausation Scores:")
for pair, value in causal_effects.items():
    print(f"Causal effect of {pair[0]} on {pair[1]}: {value}")



linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': None, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': []}
linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': None, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': []}
linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': None, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': []}
linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': None, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers': []}
linear_regression
{'control_value': 0, 'treatment_value': 1, 'test_significance': None, 'evaluate_effect_strength': False, 'confidence_intervals': False, 'target_units': 'ate', 'effect_modifiers':

In [4]:
# Convert the correlation and causation dictionaries to DataFrame for easier handling
correlation_df = pd.DataFrame(list(correlation_scores.items()), columns=['Pair', 'Correlation'])
causation_df = pd.DataFrame(list(causal_effects.items()), columns=['Pair', 'Causation'])

# Merge the two DataFrames on Pair
results_df = pd.merge(correlation_df, causation_df, on='Pair')

# Define thresholds for high and low
# These thresholds are set arbitrarily for demonstration purposes
# You might want to set these based on the data distribution like using quantiles
correlation_threshold_high = results_df['Correlation'].median()
causation_threshold_high = results_df['Causation'].median()

# Categorize each pair
results_df['Correlation_Category'] = np.where(results_df['Correlation'] >= correlation_threshold_high, 'High', 'Low')
results_df['Causation_Category'] = np.where(results_df['Causation'] >= causation_threshold_high, 'High', 'Low')

# Define categories
categories = [
    ('High', 'High'),
    ('High', 'Low'),
    ('Low', 'High'),
    ('Low', 'Low')
]

# Create a dictionary to store top 3 examples for each category
top_examples = {}

for correlation_category, causation_category in categories:
    # Filter the DataFrame for the category
    category_df = results_df[
        (results_df['Correlation_Category'] == correlation_category) &
        (results_df['Causation_Category'] == causation_category)
    ]
    
    # Sort by both correlation and causation in descending order to get top examples
    sorted_df = category_df.sort_values(by=['Correlation', 'Causation'], ascending=[False, False])
    
    # Get top 3 examples
    top_examples[(correlation_category, causation_category)] = sorted_df.head(3)

# Now print out the top examples for each category
for category, examples_df in top_examples.items():
    correlation_category, causation_category = category
    print(f"\nTop 3 examples of ({correlation_category} correlation, {causation_category} causation):")
    for index, row in examples_df.iterrows():
        print(f" - {row['Pair'][0]} -> {row['Pair'][1]} | Correlation: {row['Correlation']:.4f} | Causation: {row['Causation']:.4f}")


Top 3 examples of (High correlation, High causation):
 - review_scores_location -> review_scores_rating | Correlation: 0.9848 | Causation: 0.9845
 - bedrooms -> accommodates | Correlation: 0.8724 | Causation: 0.8724
 - accommodates -> bathrooms | Correlation: 0.8243 | Causation: 0.8243

Top 3 examples of (High correlation, Low causation):

Top 3 examples of (Low correlation, High causation):

Top 3 examples of (Low correlation, Low causation):
 - resort_access -> hot_tub | Correlation: 0.0732 | Causation: 0.0732
 - grill -> price | Correlation: 0.0705 | Causation: 0.0705
 - host_acceptance_rate -> price | Correlation: 0.0625 | Causation: 0.0625
